<h4>Import Dependencies</h4>

In [1]:
%matplotlib inline

In [8]:
import os
import csv
import pandas as pd
import random

import sqlite3
import csv
from sqlalchemy import create_engine

import matplotlib.pyplot as plt
import numpy as np

from config import pgPassword

<h4>Create a connection to SQL database</h4>

In [3]:
pg_user = 'postgres'
pg_password = pgPassword
db_name = 'baseball_db'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

<h4>Read in databases tables as DatFrames</h4>

In [4]:
teamsStatsDF = pd.read_sql_table("Team-Stats", con = engine)
battingDF = pd.read_sql_table("Batting", con = engine)
pitchingDF = pd.read_sql_table("Pitching", con = engine)
playersDF = pd.read_sql_table("Players", con = engine)
franchisesDF = pd.read_sql_table("Franchises", con = engine)
salariesDF = pd.read_sql_table("Salaries", con = engine)
teamsDF = pd.read_sql_table("Teams", con = engine)
fPlayersDF = pd.read_sql_table("FranchisePlayers", con = engine)

In [7]:
teamsStatsDF.head()

,franchiseID,yearID,G,W,R,H,HR,BB,SO,SB,RA,ERA,HA,HRA,BBA,SOA,statID,teamID
0,BNA,1871,31,20,401,426,3,60.0,19.0,73.0,303,3.55,367,2,42,23,1000,BS1-1871
1,CNA,1871,28,19,302,323,10,60.0,22.0,69.0,241,2.76,308,6,28,22,1003,CH1-1871
2,CFC,1871,29,10,249,328,7,26.0,25.0,18.0,341,4.11,346,13,53,34,1008,CL1-1871
3,KEK,1871,19,7,137,178,2,33.0,9.0,16.0,243,5.17,261,5,21,17,1015,FW1-1871
4,NNA,1871,33,16,302,403,1,33.0,15.0,46.0,313,3.72,373,7,42,22,1024,NY2-1871


In [34]:
falseOutput = teamsStatsDF[["yearID","teamID"]]
falseOutput["actual"] = teamsStatsDF.W / teamsStatsDF.G
falseOutput["model"] = ""
falseOutput["teamOnly"] = ""
for index, row in teamsStatsDF.iterrows():
    teamID = row["teamID"]
    teamOnly = teamID.split("-")[0]
    randPct = random.random()
    falseOutput.at[index, "model"] = randPct
    falseOutput.at[index, "teamOnly"] = teamOnly
falseOutput.head()

C:\Users\Vega\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Vega\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Vega\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

,yearID,teamID,actual,model,teamOnly
0,1871,BS1-1871,0.645161,0.693567,BS1
1,1871,CH1-1871,0.678571,0.123423,CH1
2,1871,CL1-1871,0.344828,0.41421,CL1
3,1871,FW1-1871,0.368421,0.641835,FW1
4,1871,NY2-1871,0.484848,0.932096,NY2


In [35]:
oneTeam = falseOutput.loc[falseOutput["teamOnly"] == "HOU"]
oneTeam.tail(10)

,yearID,teamID,actual,model,teamOnly
2636,2010,HOU-2010,0.469136,0.367285,HOU
2666,2011,HOU-2011,0.345679,0.436735,HOU
2695,2012,HOU-2012,0.339506,0.703662,HOU
2725,2013,HOU-2013,0.314815,0.0932374,HOU
2755,2014,HOU-2014,0.432099,0.941445,HOU
2785,2015,HOU-2015,0.530864,0.371035,HOU
2815,2016,HOU-2016,0.518519,0.321459,HOU
2845,2017,HOU-2017,0.623457,0.853569,HOU
2875,2018,HOU-2018,0.635802,0.389681,HOU
2905,2019,HOU-2019,0.660494,0.560695,HOU


In [65]:
modernEra = falseOutput.loc[falseOutput["yearID"] >= 1980]
# teamCodes = modernEra["teamOnly"]
# teamCodes.drop_duplicates().head(35)
modernEra = modernEra.copy()

modernEra = modernEra[modernEra.columns.drop('teamOnly')]
modernEra["model_type"] = "ML-1"
modernEra.head()

,yearID,teamID,actual,model,model_type
1787,1980,ATL-1980,0.503106,0.874219,ML-1
1788,1980,BAL-1980,0.617284,0.606125,ML-1
1789,1980,BOS-1980,0.518750,0.128914,ML-1
1790,1980,CAL-1980,0.406250,0.76688,ML-1
1791,1980,CHA-1980,0.432099,0.852062,ML-1


In [46]:
currentTeams = teamsStatsDF[["teamID", "yearID"]]
currentTeams = currentTeams.loc[currentTeams["yearID"] == 2019]
currentTeams.head(30)

,teamID,yearID
2895,ARI-2019,2019
2896,ATL-2019,2019
2897,BAL-2019,2019
2898,BOS-2019,2019
2899,CHA-2019,2019
2900,CHN-2019,2019
2901,CIN-2019,2019
2902,CLE-2019,2019
2903,COL-2019,2019
2904,DET-2019,2019


In [41]:
battingDF.head()

,yearID,stint,G,R,H,HR,BB,IBB,SO,SB,fpID
0,1954,1,35,2,10,0,3,0.0,15.0,0.0,336907024
1,1955,1,46,1,2,0,1,0.0,0.0,0.0,336907024
2,1956,1,49,4,11,0,2,0.0,28.0,0.0,336907024
3,1957,1,49,4,14,0,6,0.0,26.0,0.0,336907024
4,1958,1,46,1,6,0,5,0.0,13.0,1.0,336907024


In [67]:
latestYear = 1980
battingDF1 = battingDF.loc[battingDF["yearID"] >= latestYear]
stat = "R"
model_label = "ML-" + stat
falsePlayerOutput = battingDF1[["yearID","fpID"]]
falsePlayerOutput["actual"] = battingDF1[stat] / battingDF1.G
falsePlayerOutput["model"] = ""
for index, row in battingDF1.iterrows():
    randPct = random.random()
    falsePlayerOutput.at[index, "model"] = randPct
falsePlayerOutput["model_type"] = model_label
falsePlayerOutput.head()

C:\Users\Vega\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Vega\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Vega\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

,yearID,fpID,actual,model,model_type
87,1980,143883,0.0,0.391598,ML-R
88,1980,424800,0.0,0.816332,ML-R
89,1980,989035,0.0,0.914053,ML-R
90,1980,2809975,0.0,0.964631,ML-R
91,1980,5323248,0.0,0.846509,ML-R
